In [ ]:
import numpy as np
import pandas as pd

import os

from PIL import Image

import torch
from torch.utils.data import DataLoader, Dataset

import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms

In [ ]:
class PawpularDataset(Dataset):
    def __init__(self, images_path, features_path, transform):
        self.images_path = images_path
        self.df = pd.read_csv(features_path, dtype={'Pawpularity': np.float32})
        
        self.imgs = self.df['Id']
        self.targets = self.df['Pawpularity']
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.images_path, img_id+'.jpg')).convert('RGB')
        img = self.transform(img)
        
        targets = self.targets[index]
        
        return img, targets

In [ ]:
class PretrainedCNN(nn.Module):
    def __init__(self, hidden_size, num_classes, train=False):
        super(PretrainedCNN, self).__init__()
        self.train = train
        self.resnet = models.resnet18(pretrained=True) # TODO
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, hidden_size) # TODO
        self.fc = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, images):
        for name, param in self.resnet.named_parameters():
            if 'fc.weight' in name or 'fc.bias' in name:
                param.requires_grad = True
            else:
                param.requires_grad = self.train
                
        resnet_features = self.dropout(self.relu(self.resnet(images)))
                
        return self.fc(resnet_features)

In [ ]:
transform = transforms.Compose(
  [
   transforms.Resize((256,256)),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
  ]
)

In [ ]:
torch.manual_seed(101)

trainset = PawpularDataset(images_path='../input/petfinder-pawpularity-score/train', 
                           features_path='../input/petfinder-pawpularity-score/train.csv', 
                           transform=transform)

# TODO
# testloader = PawpularDataset(images_path='../input/petfinder-pawpularity-score/test', 
#                            features_path='../input/petfinder-pawpularity-score/test.csv', 
#                            transform=transform)
# 
# train_loader = DataLoader(
#     dataset=trainset,
#     batch_size=64,
#     shuffle=True
# )
# 
# test_loader = DataLoader(
#     dataset=testloader,
#     batch_size=64,
#     shuffle=False
# )

num_examples = len(trainset)
val_len = round(0.33*num_examples)
train_len = num_examples - val_len

train, validation = torch.utils.data.random_split(trainset, [train_len, val_len])
train_loader = torch.utils.data.DataLoader(train, batch_size=100, 
                                           shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(validation, batch_size=100, 
                                         shuffle=True, num_workers=2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hidden_size = 256
num_classes = 1
learning_rate = 3e-4
num_epochs = 10

In [ ]:
torch.manual_seed(101)

losses = list()
model = PretrainedCNN(hidden_size, num_classes).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, num_epochs+1):
    
    for step, (img, target) in enumerate(train_loader):

        model.zero_grad()

        img = img.to(device)
        target = target.to(device)

        outputs = model(img)

        loss = criterion(outputs, target.unsqueeze(1))

        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        stats = 'Epoch [%d/%d], Step [%d], Loss: %.4f' % (epoch, num_epochs, step, loss.item())
        print('\r' + stats, end='')
        
    with torch.no_grad():
        val_losses = 0
        for img, target in val_loader:
            img = img.to(device)
            target = target.to(device)
            
            outputs = model(img)
            val_loss = criterion(outputs, target.unsqueeze(1))
            val_losses += (1/len(val_loader))*val_loss.item()
            
        print('\n Epoch [%d/%d], Val Loss: %.4f' % (epoch, num_epochs, val_losses))